## MNIST with LSTM

#### 1. 导入包和数据

In [2]:
#show plt in jupyter
%matplotlib inline   

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
from PIL import Image
# import scipy
import matplotlib.pyplot as plt
import pylab

#sess 参数控制 不赘述
config = tf.ConfigProto()
sess = tf.Session(config=config)

#import mnist data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/MNIST_DATA', one_hot = True)
print(mnist.train.images.shape)

Extracting /MNIST_DATA\train-images-idx3-ubyte.gz
Extracting /MNIST_DATA\train-labels-idx1-ubyte.gz
Extracting /MNIST_DATA\t10k-images-idx3-ubyte.gz
Extracting /MNIST_DATA\t10k-labels-idx1-ubyte.gz
(55000, 784)


#### 2. 设置超参数

In [3]:
#优化算法的学习率。
#学习率决定了参数移动到最优值的速度快慢。如果学习率过大，很可能会越过最优值；反而如果学习率过小，优化的效率可能过低，长时间算法无法收敛。
learning_rate = 1e-3
# 在训练和测试的时候，我们想用不同的 batch_size.所以采用占位符的方式
batch_size = tf.placeholder(tf.int32, [])
# 每个时刻的输入特征是28维的，就是每个时刻输入一行，一行有 28 个像素
input_size = 28
# 时序持续长度为28，即每做一次预测，需要先输入28行
timestep_size = 28
# 每个隐含层的节点数
hidden_size = 64
# LSTM layer 的层数
layer_num = 2
# 最后输出分类类别数量，如果是回归预测的话应该是 1
class_num = 10

_X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, class_num])
keep_prob = tf.placeholder(tf.float32)

#### 3. 搭建LSTM模型

In [4]:
# 把784个点的字符信息还原成 28 * 28 的图片
# 下面几个步骤是实现 RNN / LSTM 的关键
X = tf.reshape(_X, [-1, 28, 28])
def unit_lstm():
    # 定义一层 LSTM_cell，只需要说明 hidden_size, 它会自动匹配输入的 X 的维度
    lstm_cell = rnn.BasicLSTMCell(num_units=hidden_size, forget_bias=1.0, state_is_tuple=True)
    #添加 dropout layer, 一般只设置 output_keep_prob
    lstm_cell = rnn.DropoutWrapper(cell=lstm_cell, input_keep_prob=1.0, output_keep_prob=keep_prob)
    return lstm_cell
#调用 MultiRNNCell 来实现多层 LSTM
mlstm_cell = rnn.MultiRNNCell([unit_lstm() for i in range(3)], state_is_tuple=True)

#用全零来初始化state
init_state = mlstm_cell.zero_state(batch_size, dtype=tf.float32)
outputs, state = tf.nn.dynamic_rnn(mlstm_cell, inputs=X, initial_state=init_state, time_major=False)
h_state = outputs[:, -1, :]  # 或者 h_state = state[-1][1]

#### 4. 设置loss function 和 optimizer

In [5]:
#tf.truncated_normal(shape, mean, stddev) :shape表示生成张量的维度，mean是均值，stddev是标准差。这个函数产生正太分布，均值和标准差自己设定。
W = tf.Variable(tf.truncated_normal([hidden_size, class_num], stddev=0.1), dtype=tf.float32)
bias = tf.Variable(tf.constant(0.1,shape=[class_num]), dtype=tf.float32)
y_pre = tf.nn.softmax(tf.matmul(h_state, W) + bias)

# 损失和评估函数
cross_entropy = -tf.reduce_mean(y * tf.log(y_pre))
train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

#### 5. 开始训练

In [6]:
#初始化所有变量
sess.run(tf.global_variables_initializer())

for i in range(1000):
    _batch_size = 128
    batch = mnist.train.next_batch(_batch_size)
    if (i+1)%200 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={
            _X:batch[0], y: batch[1], keep_prob: 1.0, batch_size: _batch_size})
        # 已经迭代完成的 epoch 数: mnist.train.epochs_completed
        print ("Iter%d, step %d, training accuracy %g" % ( mnist.train.epochs_completed, (i+1), train_accuracy))
    
    sess.run(train_op, feed_dict={_X: batch[0], y: batch[1], keep_prob: 0.5, batch_size: _batch_size})

Iter0, step 200, training accuracy 0.8125
Iter0, step 400, training accuracy 0.890625
Iter1, step 600, training accuracy 0.945312
Iter1, step 800, training accuracy 0.945312
Iter2, step 1000, training accuracy 0.960938


#### 6. 计算测试数据准确率

In [7]:
images=mnist.test.images
labels=mnist.test.labels
print ("test accuracy %g" % (sess.run(accuracy, feed_dict={
            _X:images, y: labels, keep_prob: 1.0, batch_size: mnist.test.images.shape[0]})))

test accuracy 0.9568


#### 关于MNIST 数据到底是怎样的

In [11]:
#画不出来。再想想2333
import matplotlib.pyplot as plt
current_y= mnist.train.labels[5]
current_x=mnist.train.images[5]
show(current_x)

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


NameError: name 'im' is not defined